# Word Embeddings
References: 
* https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
* https://github.com/fastai/word-embeddings-workshop/blob/master/Word%20Embeddings.ipynb

Word Embeddings are dense vectors representations of words. Compare this to sparse vectors like One-Hot Encoding.

In [6]:
sentence = "the quick brown fox jumped over the lazy dog"
words = sentence.split(' ')

Let's look at the individual words in our vocabulary:

In [7]:
vocabulary = list(set(words))
print(vocabulary)

['jumped', 'fox', 'lazy', 'brown', 'dog', 'the', 'over', 'quick']


In [8]:
# Number of words in our vocabulary
len(vocabulary)

8

# One Hot Encoding

The vocabulary size is seen above. Now we can one-hot encode the vocabulary words. Let's try using Scikit-Learn One-Hot encoding:

In [16]:
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
enc.fit(vocabulary)
enc.classes_

array(['brown', 'dog', 'fox', 'jumped', 'lazy', 'over', 'quick', 'the'],
      dtype='<U6')

Here's what our one-hot encoded vocabulary looks like.

In [18]:
enc.transform(vocabulary)

array([[0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0]])

The issue with sparse one-hot encoding is that the vectors are very large 
and we have a very sparse representation of the vectors. As you can see there
are a lot of zeroes. For example, the popular data set WikiText-103 has 267,000
words in the vocabulary. This means around 267,000 zeroes in each vector with
one-hot encoding.

We should try to find a smaller encoding for our dataset. Let's try a denser vector using
Word Em

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

In [5]:
# Training based on surrounding words. We want to predict the probability of a word based on the words around it.

# N-Gram Language Model

An N-Gram is a sequence of words as in a sentence. This is useful because it gives us some context to train a deep learning classifier. 

For a detailed post visit: https://www.microsoft.com/developerblog/2015/11/29/feature-representation-for-text-analyses/

Here's what a diagram of our n-gram deep learning model would look like:
<img src="n-gram-diagram.png">

# Continuous Bag of Words
